In [90]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

In [2]:
output_notebook()

Loading BokehJS ...

In [51]:
import pandas as pd

from bokeh.models import LinearAxis, Range1d, LabelSet, ColumnDataSource
from bokeh.layouts import gridplot, row

In [11]:
from math import sqrt

In [37]:
%ls

test-lb-policy.csv   test.csv             visualization.ipynb


In [38]:
df = pd.read_csv('test.csv')
df_lb = pd.read_csv('test-lb-policy.csv')

In [39]:
df['std_dev'] = df.var_resp_time.apply(sqrt)

In [40]:
df['small_load'] = df.small_arrival / 18.73
df_lb['small_load'] = df.small_arrival / 18.73

In [56]:
p = figure(plot_width=400, plot_height=400, title="Mean response time")
source = ColumnDataSource(df)

p.line(df.small_load, df.avg_resp_time, line_color='navy')
p.line(df_lb.small_load, df_lb.avg_resp_time, line_color='navy',line_dash="4 4", legend='LB initial policy')
p.line(df_lb.small_load, 0.9*df_lb.avg_resp_time, line_color='gray',line_dash="4 4", legend='Constraint')
p.circle(df.small_load, df.avg_resp_time, color='white', line_color='navy',size=5)

labels = LabelSet(x="small_load", y="avg_resp_time", text="beta", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)
p.grid.grid_line_dash = [2,2]
p.grid.grid_line_alpha = 0.1

p.xaxis.axis_label = 'Small cell load'
p.legend.location  = 'bottom_right'

show(p) # show the results

In [52]:
p2 = figure(plot_width=400, plot_height=400, title="Average Power")

p2.line(df.small_load, df.avg_power, line_color='firebrick')
p2.line(df_lb.small_load, df_lb.avg_power, line_color='firebrick',line_dash="4 4", legend='LB initial policy')
p2.circle(df.small_load, df.avg_power, color='white', line_color='firebrick',size=5)

labels = LabelSet(x="small_load", y="avg_power", text="beta", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')

p2.add_layout(labels)
p2.grid.grid_line_dash = [2,2]
p2.grid.grid_line_alpha = 0.1

p2.xaxis.axis_label = 'Small cell load'
p2.legend.location  = 'bottom_right'

show(p2) # show the results

In [57]:
grid = gridplot([[p,p2]])

show(grid)

In [91]:
%ls

Four_small_cells_initial_plot.png  all_runs_3
all_ru-dispatch-decisions.txt      all_runs_4
all_run-dispatch-decisions.txt     all_runs_5
all_runlog                         all_runs_6
all_runs_0                         all_runs_7
all_runs_1                         all_runs_8
all_runs_10                        all_runs_9
all_runs_11                        all_runslog
all_runs_12                        result.csv
all_runs_13                        test-lb-policy.csv
all_runs_14                        test-test.csv
all_runs_15                        test.csv
all_runs_2                         visualization.ipynb


In [322]:
df=pd.read_csv('result.csv',header=None, skiprows=24)

In [323]:
df.head(5)

,0,1,2,3,4,5,6
0,0.08437,2.0,12.0,0.08,1.0,0.86994,1191.32
1,0.15256,2.0,12.0,0.17,1.0,0.90367,1139.71
2,0.00000,2.0,9.0,0.11,1.0,0.79583,1365.62
3,0.02429,2.0,9.0,0.00,1.0,0.81674,1334.67
4,0.04060,2.0,6.0,0.00,1.0,0.90261,1154.95


In [324]:
df.rename(columns={0:'beta', 1:'macro_arrival', 2:'small_arrival', 3:'avg_idle_time', 4:'avg_setup_time',5:'avg_resp_time',6:'avg_power'},inplace=True)

In [325]:
df.sort_values(by='small_arrival', inplace=True)

In [315]:
df[5:]

,beta,relative_idle_time,small_arrival,avg_resp_time,avg_power,small_load


In [330]:
import numpy as np
for i in range(28):
    if i <16:
        r=0
    else:
        r=1
    df_tmp = pd.read_csv('all_runs_'+str(i),skiprows=r,usecols=['beta','small_arrival','avg_idle_time','avg_power','avg_resp_time'],dtype={'beta':np.float64, 'small_arrival':np.float64,'avg_idle_time':np.float64,'avg_power':np.float64})
    df = df.append(df_tmp.loc[0])

In [331]:
df['relative_idle_time'] = df.small_arrival * df.avg_idle_time

In [332]:
df['relative_idle_time']=df.relative_idle_time.apply(round)

In [333]:
import copy
df = copy.copy(df[['beta','relative_idle_time','small_arrival','avg_resp_time','avg_power']])

In [334]:
df['small_load'] = df.small_arrival / 18.73
df.head(5)

,beta,relative_idle_time,small_arrival,avg_resp_time,avg_power,small_load
13,0.00000,0,4.0,0.75441,1281.31,0.213561
10,0.80785,2,4.0,0.69621,1073.76,0.213561
9,1.45908,6,4.0,0.46607,1065.65,0.213561
8,1.21848,4,4.0,0.55521,1068.75,0.213561
7,1.62559,8,4.0,0.40263,1063.25,0.213561


In [338]:
df['idx'] = list(range(56))
df.set_index('idx',inplace=True)

In [354]:
df.beta = df.beta.apply(lambda x: round(x, 4))

In [361]:

def filtered(arr, betazero=False):
    
    d = copy.copy(df)
    if betazero == False:
        #d = d[d['beta'] != 0.00000]
        d = d.loc[:27]
    else:
        #d = d[d['beta'] == 0.00000]
        d = d.loc[28:]
    d=d[d['small_arrival']==arr]
    d.sort_values(by='relative_idle_time',inplace=True)
    return d
p0 = figure(plot_width=600, plot_height=600, title="Mean response time")

source = ColumnDataSource(df.loc[0:27])

p0.line(filtered(4.0).relative_idle_time, filtered(4.0).avg_resp_time, line_color='navy',legend='Small cell load = 0.2')
p0.circle(filtered(4.0).relative_idle_time, filtered(4.0).avg_resp_time, color='white', line_color='navy',size=5)

p0.line(filtered(6.0).relative_idle_time, filtered(6.0).avg_resp_time, line_color='navy',line_dash="8 8", legend='                        = 0.32')
p0.circle(filtered(6.0).relative_idle_time, filtered(6.0).avg_resp_time, color='white', line_color='navy',size=5)

p0.line(filtered(9.0).relative_idle_time, filtered(9.0).avg_resp_time, line_color='navy',line_dash="4 4", legend='                        = 0.5')
p0.circle(filtered(9.0).relative_idle_time, filtered(9.0).avg_resp_time, color='white', line_color='navy',size=5)

p0.line(filtered(12.0).relative_idle_time, filtered(12.0).avg_resp_time, line_color='navy',line_dash="2 6", legend='                        = 0.65')
p0.circle(filtered(12.0).relative_idle_time, filtered(12.0).avg_resp_time, color='white', line_color='navy',size=5)

p0.line([0,1,2,10],[0.934192,0.934192,0.934192,0.934192],line_color='gray',legend='constraint')

#labels = LabelSet(x="relative_idle_time", y="avg_resp_time", text="beta", y_offset=8,
#                  text_font_size="8pt", text_color="#555555",
#                  source=source, text_align='center')
#p0.add_layout(labels)
p0.grid.grid_line_dash = [2,2]
p0.grid.grid_line_alpha = 0.1

p0.xaxis.axis_label = 'idle time (relative to arrival rate)'
p0.legend.location  = 'top_right'

show(p0) # show the results

In [348]:
filtered(4)

,beta,relative_idle_time,small_arrival,avg_resp_time,avg_power,small_load
idx,,,,,,
0,0.00000,0,4.0,0.75441,1281.31,0.213561
6,0.50262,1,4.0,0.80249,1077.59,0.213561
1,0.80785,2,4.0,0.69621,1073.76,0.213561
3,1.21848,4,4.0,0.55521,1068.75,0.213561
2,1.45908,6,4.0,0.46607,1065.65,0.213561
4,1.62559,8,4.0,0.40263,1063.25,0.213561
5,1.75193,10,4.0,0.35863,1061.62,0.213561


In [360]:
p = figure(plot_width=600, plot_height=700, title="Power")
source = ColumnDataSource(df)

p.line(filtered(4.0).relative_idle_time, filtered(4.0).avg_power, line_color='navy',legend='Small cell load = 0.2')
p.circle(filtered(4.0).relative_idle_time, filtered(4.0).avg_power, color='white', line_color='navy',size=5)

p.line(filtered(6.0).relative_idle_time, filtered(6.0).avg_power, line_color='navy',line_dash="8 8", legend='                        = 0.32')
p.circle(filtered(6.0).relative_idle_time, filtered(6.0).avg_power, color='white', line_color='navy',size=5)

p.line(filtered(9.0).relative_idle_time, filtered(9.0).avg_power, line_color='navy',line_dash="4 4", legend='                        = 0.5')
p.circle(filtered(9.0).relative_idle_time, filtered(9.0).avg_power, color='white', line_color='navy',size=5)

p.line(filtered(12.0).relative_idle_time, filtered(12.0).avg_power, line_color='navy',line_dash="2 6", legend='                        = 0.65')
p.circle(filtered(12.0).relative_idle_time, filtered(12.0).avg_power, color='white', line_color='navy',size=5)

p.line(filtered(4.0,True).relative_idle_time, filtered(4.0,True).avg_power, line_color='firebrick')
p.circle(filtered(4.0, True).relative_idle_time, filtered(4.0,True).avg_power, color='white', line_color='firebrick',size=5)

p.line(filtered(6.0,True).relative_idle_time, filtered(6.0,True).avg_power, line_dash="8 8",line_color='firebrick')
p.circle(filtered(6.0, True).relative_idle_time, filtered(6.0,True).avg_power, color='white', line_color='firebrick',size=5)

p.line(filtered(9.0,True).relative_idle_time, filtered(9.0,True).avg_power, line_dash="4 4",line_color='firebrick')
p.circle(filtered(9.0, True).relative_idle_time, filtered(9.0,True).avg_power, color='white', line_color='firebrick',size=5)

p.line(filtered(12.0,True).relative_idle_time, filtered(12.0,True).avg_power, line_dash="2 6",line_color='firebrick')
p.circle(filtered(12.0, True).relative_idle_time, filtered(12.0,True).avg_power, color='white', line_color='firebrick',size=5)

#labels = LabelSet(x="relative_idle_time", y="avg_power", text="beta", y_offset=8,
#                  text_font_size="8pt", text_color="#555555",
#                  source=source, text_align='center')
#p.add_layout(labels)

p.grid.grid_line_dash = [2,2]
p.grid.grid_line_alpha = 0.1

p.xaxis.axis_label = 'idle time (relative to arrival rate)'
p.legend.location  = 'top_right'

show(p) # show the results

In [282]:
df

,beta,relative_idle_time,small_arrival,avg_resp_time,avg_power,small_load
13,0.00000,0,4.0,0.75441,1281.31,0.213561
10,0.80785,2,4.0,0.69621,1073.76,0.213561
9,1.45908,6,4.0,0.46607,1065.65,0.213561
8,1.21848,4,4.0,0.55521,1068.75,0.213561
7,1.62559,8,4.0,0.40263,1063.25,0.213561
6,1.75193,10,4.0,0.35863,1061.62,0.213561
11,0.50262,1,4.0,0.80249,1077.59,0.213561
4,0.04060,0,6.0,0.90261,1154.95,0.320342
12,1.48045,10,6.0,0.44747,1081.16,0.320342
14,1.34788,8,6.0,0.49293,1083.25,0.320342


In [182]:
df.loc[2,'avg_power'] = 1184

/Users/misikir/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [191]:
grid = gridplot([[p0,p]])

show(grid)

In [545]:
df=pd.read_csv('output_0.csv', skiprows=1)

In [546]:
df_rnd = df.loc[:0]
df = df.loc[20:]

In [547]:
for i in range(1,32):
    df_tmp = pd.read_csv('output_'+str(i)+'.csv', skiprows=1)
    df_rnd_tmp = df_tmp.loc[0,:]
    df_rnd = df_rnd.append(df_rnd_tmp)
    df_tmp = df_tmp.iloc[-1:]
    df = df.append(df_tmp)

In [548]:
df

,beta,macro_arrival,small_arrival,avg_idle_time,avg_setup_time,num_of_jobs,avg_resp_time,var_resp_time,avg_power
20,2.68971,2.0,1.0,0.00,20.0,3798953,0.09791,0.00649,784.92
20,2.74700,2.0,1.0,1.00,20.0,3800227,0.08178,0.00596,905.78
20,2.76727,2.0,1.0,2.00,20.0,3798696,0.07604,0.00561,947.10
20,2.78381,2.0,1.0,4.00,20.0,3798974,0.07137,0.00527,980.67
20,2.79513,2.0,1.0,8.00,20.0,3800281,0.06820,0.00501,1003.43
20,2.80790,2.0,1.0,128.00,20.0,3800335,0.06459,0.00470,1030.07
20,2.80877,2.0,1.0,512.00,20.0,3799560,0.06435,0.00467,1031.52
20,2.80886,2.0,1.0,1024.00,20.0,3796545,0.06432,0.00467,1031.84
20,2.65562,2.0,4.0,0.00,20.0,3797474,0.10767,0.00809,880.91
20,2.72615,2.0,4.0,0.25,20.0,3797997,0.08773,0.00709,958.55


In [549]:
df['relative_idle_time'] = df.small_arrival * df.avg_idle_time
df.relative_idle_time    = df.relative_idle_time.apply(lambda x: round(x))

df_rnd['relative_idle_time'] = df_rnd.small_arrival * df_rnd.avg_idle_time
df_rnd.relative_idle_time    = df_rnd.relative_idle_time.apply(lambda x: round(x))

In [550]:
df_rel = pd.DataFrame()
df_rel['relative_idle_time'] = df.relative_idle_time.apply(lambda x: np.log2(x))
df_rel['small_arrival'] = df.small_arrival
df_rel['relative_power'] = [df.avg_power.iloc[i] / df_rnd.avg_power.iloc[i] for i in range(32)]

In [565]:
def logscale(arrival):
    rel_idle = df[df.small_arrival==arrival].relative_idle_time
    rel_idle = rel_idle.apply(lambda x: np.log2(x)).tolist()
    rel_idle.pop(0)
    rel_idle.insert(0, -10)
    
    return rel_idle

In [566]:
p0 = figure(plot_width=600, plot_height=600, title="Mean response time")

#source = ColumnDataSource(df.loc[0:27])

p0.line(logscale(1), df[df.small_arrival==1].avg_resp_time, line_color='navy',legend='Small cell load = 0.05')
p0.circle(logscale(1), df[df.small_arrival==1].avg_resp_time, color='white', line_color='navy',size=5)
p0.line(logscale(4), df[df.small_arrival==4].avg_resp_time, line_color='navy',line_dash="15 2",legend='Small cell load = 0.2')
p0.circle(logscale(4), df[df.small_arrival==4].avg_resp_time, color='white', line_color='navy',size=5)

p0.line(logscale(6), df[df.small_arrival==6].avg_resp_time, line_color='navy',line_dash="4 4",legend='Small cell load = 0.32')
p0.circle(logscale(6), df[df.small_arrival==6].avg_resp_time, color='white', line_color='navy',size=5)

p0.line(logscale(9), df[df.small_arrival==9].avg_resp_time, line_color='navy',line_dash="1 4",legend='Small cell load = 0.5')
p0.circle(logscale(9), df[df.small_arrival==9].avg_resp_time, color='white', line_color='navy',size=5)



p0.line([-10,0,1,2,10],0.1350*0.85*np.ones(5),line_color='gray',legend='constraint')

p0.grid.grid_line_dash = [2,2]
p0.grid.grid_line_alpha = 0.1

p0.xaxis.axis_label = 'idle time in logscale (relative to avg. avarrival time)'
p0.legend.location  = 'top_right'

show(p0)

In [567]:
p0 = figure(plot_width=600, plot_height=600, title="Power")


p0.line(logscale(1), df[df.small_arrival==1].avg_power, line_color='navy',legend='Small cell load = 0.05')
p0.circle(logscale(1), df[df.small_arrival==1].avg_power, color='white', line_color='navy',size=5)
p0.line(logscale(4), df[df.small_arrival==4].avg_power, line_color='navy',line_dash="15 2",legend='Small cell load = 0.2')
p0.circle(logscale(4), df[df.small_arrival==4].avg_power, color='white', line_color='navy',size=5)

p0.line(logscale(6), df[df.small_arrival==6].avg_power, line_color='navy',line_dash="4 4",legend='Small cell load = 0.32')
p0.circle(logscale(6), df[df.small_arrival==6].avg_power, color='white', line_color='navy',size=5)

p0.line(logscale(9), df[df.small_arrival==9].avg_power, line_color='navy',line_dash="1 4",legend='Small cell load = 0.5')
p0.circle(logscale(9), df[df.small_arrival==9].avg_power, color='white', line_color='navy',size=5)


p0.grid.grid_line_dash = [2,2]
p0.grid.grid_line_alpha = 0.1

p0.xaxis.axis_label = 'idle time in logscale (relative to avg. avarrival time)'
p0.legend.location  = 'bottom_right'

show(p0)

In [499]:
p0 = figure(plot_width=600, plot_height=600, title="Power")


p0.line(df_rel[df_rel.small_arrival==1].relative_idle_time, df_rel[df_rel.small_arrival==1].relative_power, line_color='navy',legend='Small cell load = 0.05')
p0.circle(df_rel[df_rel.small_arrival==1].relative_idle_time, df_rel[df_rel.small_arrival==1].relative_power, color='white', line_color='navy',size=5)

p0.line(df_rel[df_rel.small_arrival==4].relative_idle_time, df_rel[df_rel.small_arrival==4].relative_power, line_dash="15 2",line_color='navy',legend='Small cell load = 0.2')
p0.circle(df_rel[df_rel.small_arrival==4].relative_idle_time, df_rel[df_rel.small_arrival==4].relative_power, color='white', line_color='navy',size=5)

p0.line(df_rel[df_rel.small_arrival==6].relative_idle_time, df_rel[df_rel.small_arrival==6].relative_power, line_dash="4 4",line_color='navy',legend='Small cell load = 0.32')
p0.circle(df_rel[df_rel.small_arrival==6].relative_idle_time, df_rel[df_rel.small_arrival==6].relative_power, color='white', line_color='navy',size=5)

p0.line(df_rel[df_rel.small_arrival==9].relative_idle_time, df_rel[df_rel.small_arrival==9].relative_power, line_dash="1 4",line_color='navy',legend='Small cell load = 0.5')
p0.circle(df_rel[df_rel.small_arrival==9].relative_idle_time, df_rel[df_rel.small_arrival==9].relative_power, color='white', line_color='navy',size=5)

p0.grid.grid_line_dash = [2,2]
p0.grid.grid_line_alpha = 0.1

p0.xaxis.axis_label = 'idle time in logscale (relative to avg. avarrival time)'
p0.legend.location  = 'top_right'

show(p0)

In [501]:
%cat output_56.csv


beta,macro_arrival,small_arrival,avg_idle_time,avg_setup_time,num_of_jobs,avg_resp_time,var_resp_time,avg_power
0.00000,2.00,9.00,0.00,20.00,3799162,0.13501,0.02038,1237.53
0.00000,2.00,9.00,0.00,20.00,3801671,0.12418,0.01394,1240.18
-0.00751,2.00,9.00,0.00,20.00,3803416,0.13665,0.02051,1224.80
-0.01750,2.00,9.00,0.00,20.00,3800281,0.17080,0.05084,1221.77
-0.03555,2.00,9.00,0.00,20.00,3800970,0.24213,0.18028,1219.87
-0.06691,2.00,9.00,0.00,20.00,3798900,0.36446,0.61761,1223.50
-0.11647,2.00,9.00,0.00,20.00,3797694,0.55706,1.86809,1225.83
-0.18987,2.00,9.00,0.00,20.00,3800672,0.84477,5.01232,1227.14
-0.29388,2.00,9.00,0.00,20.00,3802788,1.25171,12.04604,1228.17
-0.43576,2.00,9.00,0.00,20.00,3799990,1.81040,26.63029,1228.23
-0.62395,2.00,9.00,0.00,20.00,3801349,2.54922,54.64924,1228.48
-0.86721,2.00,9.00,0.00,20.00,3799034,3.50855,106.18122,1228.37


# Idle power the same as busy power

In [534]:
df=pd.read_csv('output_32.csv', skiprows=1)
df = df.loc[20:]

for i in range(33,64):
    df_tmp = pd.read_csv('output_'+str(i)+'.csv', skiprows=1)
    df_tmp = df_tmp.iloc[-1]
    df = df.append(df_tmp)

In [535]:
df['relative_idle_time'] = df.small_arrival * df.avg_idle_time
df.relative_idle_time    = df.relative_idle_time.apply(lambda x: round(x))

In [543]:
p0 = figure(plot_width=600, plot_height=600, title="Mean response time")

#source = ColumnDataSource(df.loc[0:27])

p0.line(logscale(1), df[df.small_arrival==1].avg_resp_time, line_color='navy',legend='Small cell load = 0.05')
p0.circle(logscale(1), df[df.small_arrival==1].avg_resp_time, color='white', line_color='navy',size=5)
p0.line(logscale(4), df[df.small_arrival==4].avg_resp_time, line_color='navy',line_dash="15 2",legend='Small cell load = 0.2')
p0.circle(logscale(4), df[df.small_arrival==4].avg_resp_time, color='white', line_color='navy',size=5)

p0.line(logscale(6), df[df.small_arrival==6].avg_resp_time, line_color='navy',line_dash="4 4",legend='Small cell load = 0.32')
p0.circle(logscale(6), df[df.small_arrival==6].avg_resp_time, color='white', line_color='navy',size=5)

p0.line(logscale(9), df[df.small_arrival==9].avg_resp_time.tolist(), line_color='navy',line_dash="1 4",legend='Small cell load = 0.5')
p0.circle(logscale(9), df[df.small_arrival==9].avg_resp_time, color='white', line_color='navy',size=5)



p0.line([0,1,2,10],0.1350*0.85*np.ones(4),line_color='gray',legend='constraint')

p0.grid.grid_line_dash = [2,2]
p0.grid.grid_line_alpha = 0.1

p0.xaxis.axis_label = 'idle time in logscale (relative to avg. avarrival time)'
p0.legend.location  = 'top_right'

show(p0)

In [525]:
p0 = figure(plot_width=600, plot_height=600, title="Power")


p0.line(logscale(1), df[df.small_arrival==1].avg_power, line_color='navy',legend='Small cell load = 0.05')
p0.circle(logscale(1), df[df.small_arrival==1].avg_power, color='white', line_color='navy',size=5)
p0.line(logscale(4), df[df.small_arrival==4].avg_power, line_color='navy',line_dash="15 2",legend='Small cell load = 0.2')
p0.circle(logscale(4), df[df.small_arrival==4].avg_power, color='white', line_color='navy',size=5)

p0.line(logscale(6), df[df.small_arrival==6].avg_power, line_color='navy',line_dash="4 4",legend='Small cell load = 0.32')
p0.circle(logscale(6), df[df.small_arrival==6].avg_power, color='white', line_color='navy',size=5)

p0.line(logscale(9), df[df.small_arrival==9].avg_power, line_color='navy',line_dash="1 4",legend='Small cell load = 0.5')
p0.circle(logscale(9), df[df.small_arrival==9].avg_power, color='white', line_color='navy',size=5)


p0.grid.grid_line_dash = [2,2]
p0.grid.grid_line_alpha = 0.1

p0.xaxis.axis_label = 'idle time in logscale (relative to avg. avarrival time)'
p0.legend.location  = 'bottom_right'

show(p0)

In [537]:
df

,beta,macro_arrival,small_arrival,avg_idle_time,avg_setup_time,num_of_jobs,avg_resp_time,var_resp_time,avg_power,relative_idle_time
20,0.06647,2.0,1.0,0.00,20.0,3800886.0,0.09795,0.00651,1038.99,0
20,0.12390,2.0,1.0,1.00,20.0,3800578.0,0.08176,0.00596,1159.67,1
20,0.14423,2.0,1.0,2.00,20.0,3798389.0,0.07604,0.00561,1201.16,2
20,0.16069,2.0,1.0,4.00,20.0,3800151.0,0.07138,0.00527,1234.87,4
20,0.17201,2.0,1.0,8.00,20.0,3799940.0,0.06817,0.00500,1257.60,8
20,0.18500,2.0,1.0,128.00,20.0,3801120.0,0.06451,0.00468,1284.27,128
20,0.18574,2.0,1.0,512.00,20.0,3796671.0,0.06430,0.00466,1285.76,512
20,0.18582,2.0,1.0,1024.00,20.0,3799551.0,0.06429,0.00467,1286.02,1024
20,0.03228,2.0,4.0,0.00,20.0,3800287.0,0.10772,0.00807,1134.10,0
20,0.10319,2.0,4.0,0.25,20.0,3800342.0,0.08767,0.00708,1211.43,1


In [542]:
df[df.small_arrival==9].avg_resp_time.tolist()

[0.12418,
 0.11,
 0.10147,
 0.09376,
 0.08769,
 0.08069,
 0.08012999999999999,
 0.08005]

In [553]:
logscale(9)